In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf
import re

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select\
                      x.q_id\
                    , x.q_posttypeid\
                    , x.q_acceptedanswerid\
                    , x.q_parentid\
                    , x.q_creationdate\
                    , x.q_score\
                    , x.q_viewcount\
                    , x.q_owneruserid\
                    , x.q_title\
                    , x.q_tags\
                    , x.q_answercount\
                    , x.q_commentcount\
                    , x.q_reputation\
                    , f.body             as q_body\
                    , b.id               as a_id\
                    , b.posttypeid       as a_posttypeid\
                    , b.acceptedanswerid as a_acceptedanswerid\
                    , b.parentid         as a_parentid\
                    , b.creationdate     as a_creationdate\
                    , b.score            as a_score\
                    , b.viewcount        as a_viewcount\
                    , b.owneruserid      as a_owneruserid\
                    , b.title            as a_title\
                    , b.tags             as a_tags\
                    , b.answercount      as a_answercount\
                    , b.commentcount     as a_commentcount\
                    , d.reputation       as a_reputation\
                    , g.body             as a_body\
      from (\
               select a.id               as q_id\
                    , a.posttypeid       as q_posttypeid\
                    , a.acceptedanswerid as q_acceptedanswerid\
                    , a.parentid         as q_parentid\
                    , a.creationdate     as q_creationdate\
                    , a.score            as q_score\
                    , a.viewcount        as q_viewcount\
                    , a.owneruserid      as q_owneruserid\
                    , a.title            as q_title\
                    , a.answercount      as q_answercount\
                    , a.commentcount     as q_commentcount\
                    , c.reputation       as q_reputation\
                    , replace(replace(lower(a.tags), '<', ''), '>', ' ')as q_tags\
               from posts a\
                  , users c\
               where a.creationdate >= '2023-11-17'\
                 and a.posttypeid = '1'\
                 and a.owneruserid is not null\
                 and c.id = a.owneruserid\
                 and (replace(replace(lower(a.tags), '<', ''), '>', ' ') like '%python %')\
           )   x\
            , posts b\
            , users d\
            , postsbody f\
            , postsbody g\
    where b.parentid = x.q_id\
      and b.posttypeid = '2'\
      and b.owneruserid is not null\
      and d.id = b.owneruserid\
      and x.q_id = f.id \
      and b.id = g.id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [ ]:
df = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_posttypeid' 
, 'q_acceptedanswerid'
, 'q_parentid' 
, 'q_creationdate' 
, 'q_score' 
, 'q_viewcount' 
, 'q_owneruserid' 
, 'q_title' 
, 'q_tags' 
, 'q_answercount' 
, 'q_commentcount' 
, 'q_reputation' 
, 'q_text' 
, 'a_id' 
, 'a_posttypeid' 
, 'a_acceptedanswerid' 
, 'a_parentid' 
, 'a_creationdate' 
, 'a_score' 
, 'a_viewcount' 
, 'a_owneruserid' 
, 'a_title' 
, 'a_tags' 
, 'a_answercount'
, 'a_commentcount'
, 'a_reputation' 
, 'a_text' 
])

In [ ]:
# 전체 데이터셋 
with open('../../data/bert_df.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
print(df['q_creationdate'].min())
print(df['a_creationdate'].max())

In [3]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select a.q_id, a.a_id, a.q_creationdate, c.tags, b.body \
  from ( \
  select q_id, q_creationdate, max(a_id) as a_id \
  from tt_qna_240506 \
 group by q_id, q_creationdate ) a \
     , postsbody b \
     , posts c \
where a.q_id = b.id \
  and a.q_id = c.id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [4]:
df = pd.DataFrame(rows, columns = [
  'q_id' 
, 'a_id'
, 'q_creationdate'
, 'tags' 
, 'body' 
])

In [5]:
# 전체 데이터셋 
with open('../../data/bert_src_df.pkl', 'wb') as f:
    pickle.dump(df, f)

In [ ]:
# create table tt_qna_240506 as
# select 
#                   x.q_id 
#                 , x.q_creationdate 
#                 , b.id               as a_id 
#   from ( 
#            select a.id               as q_id 
#                 , a.creationdate     as q_creationdate 
#            from posts a 
#               , users c 
#           where a.creationdate between '2020-11-01' and '2023-11-30' 
#              and a.posttypeid = '1' 
#              and a.owneruserid is not null 
#              and c.id = a.owneruserid 
#        )   x 
# LEFT JOIN posts b ON 
# 	x.q_id = b.parentid 
# ;

In [ ]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select a.q_id, a.a_id, a.q_creationdate, c.tags, b.body \
  from tt_qna_240506 a \
     , postsbody b \
     , posts c \
where a.q_id = b.id \
  and a.q_id = c.id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [6]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select b.owneruserid as ans_usr_id \
     , a.creationdate as q_date \
     , b.creationdate as a_date \
	 , a.id as q_id \
	 , b.id as a_id \
     , c.body as question_body \
     , d.body as answer_body \
  from posts a \
     , posts b \
     , postsbody c \
     , postsbody d \
 where a.creationdate between '20221101' and '20221231' \
   and a.posttypeid = '1' \
   and a.id = b.parentid \
   and a.id = c.id \
   and b.id = d.id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()



In [8]:
df = pd.DataFrame(rows, columns = [
  'ans_usr_id' 
, 'q_creationdate'
, 'a_creationdate'
, 'q_id'
, 'a_id'
, 'question_body' 
, 'answer_body' 
])

In [9]:
# 전체 데이터셋 
with open('../../data/bert_qna_body.pkl', 'wb') as f:
    pickle.dump(df, f)